# Программирование для всех

*Алла Тамбовцева, НИУ ВШЭ*

## Парсинг HTML-файлов с помощью `BeautifulSoup`

Мы уже немного познакомились со структурой html-страниц, теперь попробуем выгрузить информацию из реально существующей страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main`:

In [2]:
main = "https://nplus1.ru/"

Сформируем запрос, отправим его и получим ответ с помощью функции `get()` из `requests`:

In [3]:
page = requests.get(main) 

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [4]:
page

<Response [200]>

Объект page имеет тип `Response` и скрыт от наших глаз. Однако при его вызове мы видим число 200 – это код результата, который означает, что страница благополучно загружена.

У объекта типа `Response` есть атрибут `.text`, в котором хранится исходный код страницы, который мы можем посмотреть, нажав *Ctrl+U* в Chrome:

In [5]:
page.text

'<!DOCTYPE html>\n<html prefix="og: http://ogp.me/ns#" lang="ru">\n<head>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <title>N + 1: научные статьи, новости, открытия</title>\n  <link href="/front-build/css/main.css" rel="stylesheet">\n  <link href="/front-build/css/app.css?v=1" rel="stylesheet">\n  \n  <link rel="icon" href="/images-new/favicon-bw.png"/>\n\n  <link rel="canonical" href="https://nplus1.ru">\n\n  <meta name="yandex-verification" content="8c90b02c84ac3b72"/>\n  <meta name="pmail-verification" content="b419949322895fc9106e24ed01be58ac">\n\n  <script>window.yaContextCb = window.yaContextCb || []</script>\n  <script src="https://yandex.ru/ads/system/context.js" async></script>\n\n  \n\n  <meta property="og:site_name" content="N + 1: научные статьи, новости, открытия"/>\n  <meta property="og:title" content="N + 1: научные статьи, новости, открытия"/>\n  <meta property="og:image" content="https://nplus1.ru/images-new/sha

Результат выше – это обычная строка, тип string. Выполнять поиск по такой строке неудобно, поэтому преобразуем эту строку в объект типа `BeautifulSoup`, который позволяет выполнять поиск по тегам:

In [6]:
soup = BeautifulSoup(page.text)

In [7]:
type(soup)

bs4.BeautifulSoup

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [8]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

В коде выше мы использовали метод `.find_all()`, который выполняет поиск по заданному тэгу и возвращает список частей кода HTML с выбранным тэгом. Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.

Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [9]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [10]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li) 

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [11]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [12]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2022/11/03/ancient-dna


Отправим запрос к этой странице с одной новостью и обработаем исходный код страницы:

In [13]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

Попробуем найти заголовок новости. Изучив исходный код страницы, заметим, что он находится в разных местах страницы. Возьмем самый простой вариант – тэг `<title>`:

In [14]:
# find() вместо find_all(), так как результат точно будет один
# незачем писать soup0.find_all("title")[0] для извлечения единственного элемента списка,
# если можно сразу вернуть этот единственный элемент

soup0.find("title")

<title>Население Южной Америки разделилось на две группы не менее десяти тысяч лет назад</title>

Извлечем текст, который заключен внутри тэгов:

In [15]:
title = soup0.find("title").text
title

'Население Южной Америки разделилось на\xa0две группы не\xa0менее десяти тысяч лет назад'

Найдем теперь имя автора. Заметим, что оно находится в тэге `<meta>`, причем не просто в `<meta>`, а с атрибутом `name`, равным `mediator_author`. Чтобы учесть это уточнение в поиске, подадим на вход `.find()` словарь с названием атрибута и его значением:

In [16]:
soup0.find("meta", {"name" : "mediator_author"})

<meta content="Михаил Подрезов" name="mediator_author"/>

Осталось извлечь имя автора из `content` по аналогии с тем, как мы ранее извлекали ссылку из `href`. Используем метод `.get()`:

In [17]:
author = soup0.find("meta", {"name" : "mediator_author"}).get("content") 

Аналогичным образом извлечем дату публикации новости:

In [18]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 

С остальными характеристиками – сложностью и рубриками – все более интересно. С одной стороны, их можно найти точно так же по тэгам и атрибутам, но для надежности (не все названия атрибутов здесь понятны и гарантируют уникальность) давайте сузим пространство для поиска и будем искать их не по всей странице, а в пределах какого-нибудь раздела. Такой раздел есть, он имеет очень длинное значение атрибута `class`:

In [19]:
div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
div

<div class="flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9">
<span class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center">
<span class="group-hover:text-main transition-colors duration-75">10:02</span>
</span>
<a class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center" href="/news/2022/11/03">
<span class="group-hover:text-main transition-colors duration-75">03.11.22</span>
</a>
<a class="relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center" href="/material/difficulty/3">
<svg class="w-4 h-4 mr-1 group-hover:text-main transition-colors duration-75 stroke-current">
<use xlink:href="#n1_star"></use>
</svg>
<span class="group-hover:text-main transition-colors duration-75">3.9</span>
</a>
<a class="relative before:block before:w-px before:b

В этом разделе встречаются более мелкие части с тэгами `span`, в которых хранится дата, сложность статьи и рубрики. К сожалению, сайт сейчас в процессе перехода к новой структуре и новому дизайну, поэтому не всегда последовательность элементов будет такой (где-то будет вклиниваться время публикации статьи), но каким-то быстрым способом мы с этой проблемой сейчас не справимся. Поэтому отберем элементы как есть:

In [20]:
diffc = div.find_all("span")[1]
rubs_raw = div.find_all("span")[2:]

In [21]:
print(diffc)
print(rubs_raw)

<span class="group-hover:text-main transition-colors duration-75">10:02</span>
[<span class="group-hover:text-main transition-colors duration-75">03.11.22</span>, <span class="group-hover:text-main transition-colors duration-75">3.9</span>, <span class="group-hover:text-main transition-colors duration-75">Антропология</span>, <span class="group-hover:text-main transition-colors duration-75">Археология</span>]


Теперь давайте извлечем из блоков кода HTML с рубриками текст с названиями рубрик:

In [22]:
rubs = [r.text for r in rubs_raw]
rubs

['03.11.22', '3.9', 'Антропология', 'Археология']

Склеим их в одну строку по запятой:

In [23]:
rubs_text = ", ".join(rubs)

Осталось аналогичным образом сохранить самое главное – текст новости. Заметим, что его нужно собирать по абзацам – по тэгам `<p>` с атрибутом `class`, равным `mb-6`:

In [24]:
pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars_raw

[<p class="text-36 md:text-44 xl:text-54 font-spectral text-main-gray mb-6">Похоже, это произошло в андском регионе</p>,
 <p class="mb-6">Палеогенетики секвенировали древнюю ДНК двух людей, останки которых нашли в Бразилии. Объединив полученные данные с ранее опубликованными геномами, они пришли к выводу, что как минимум десять тысяч лет назад древнейшее население Южной Америки разделилось на две популяции. Вероятно, это событие произошло в андском регионе. После чего одна часть дала начало населению Южного конуса, а другая — популяциям атлантического побережья. Результаты исследования <a href="https://doi.org/10.1098/rspb.2022.1078">опубликованы</a> в журнале <em>Proceedings of the Royal Society B: Biological Sciences</em>.</p>,
 <p class="mb-6">Время и маршруты заселения Америки — тема весьма дискуссионная. На протяжении долгого времени древнейшим населением этого континента считались представители археологической <a href="https://nplus1.ru/news/2021/11/22/clovis-culture">культуры Кл

Извлечем текст в чистом виде из каждого элемента списка `pars_raw`:

In [25]:
pars = [p.text for p in pars_raw] 

Склеим полученные строки в одну большую строку через `.join()`, а заодно избавимся от посторонних символов `\xa0`, которые соответствуют неразрывным пробелам, и `\n`, которые соответствуют переходам на новую строку:

In [26]:
text = " ".join(pars)
text = text.replace("\xa0", " ") 
text = text.replace("\n", " ") 

Теперь все красиво. Напишем функцию для выгрузки информации по одной новости и применим ее к новостям на главной странице.

In [27]:
def get_news(link0):
    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text) 
    title = soup0.find("title").text
    author = soup0.find("meta", {"name" : "mediator_author"}).get("content") 
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    div = soup0.find("div",
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[1].text
    rubs_raw = div.find_all("span")[2:]
    rubs = [r.text for r in rubs_raw]
    rubs_text = ", ".join(rubs)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ") 
    return title, author, date, diffc, rubs_text, text

Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [28]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [29]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.5)

https://nplus1.ru/news/2022/11/03/ancient-dna
https://nplus1.ru/news/2022/11/03/Truly-chiral
https://nplus1.ru/news/2022/11/03/children-videogames
https://nplus1.ru/news/2022/11/03/height-caffeine
https://nplus1.ru/news/2022/11/03/ML-Suzuki
https://nplus1.ru/news/2022/11/03/melanoplus-spretus
https://nplus1.ru/news/2022/11/03/cats-DNA-human
https://nplus1.ru/news/2022/11/03/ancient-dna
https://nplus1.ru/news/2022/11/02/xiaomi-leica
https://nplus1.ru/news/2022/11/02/over6mets-mortality
https://nplus1.ru/news/2022/11/02/heating-europe
https://nplus1.ru/news/2022/11/02/historic-cache-in-ice
https://nplus1.ru/news/2022/11/02/vortex-stripes
https://nplus1.ru/news/2022/11/02/vampire-from-usa
https://nplus1.ru/news/2022/11/02/slavic-hillfort
https://nplus1.ru/news/2022/11/01/heavy-4
https://nplus1.ru/news/2022/11/01/birds-size
https://nplus1.ru/news/2022/11/01/xenicus-gilviventris
https://nplus1.ru/news/2022/11/01/2022-ap-7
https://nplus1.ru/news/2022/11/01/ecg-swine2human
https://nplus1.ru/n

KeyboardInterrupt: 

Ура! Несмотря на то, что ближе к концу списка со ссылками мы стали получать сообщения об ошибках, новости за последние несколько дней сгрузились корректно (сайт в разработке, пока небольшие проблемы с унификацией и упорядочиванием ссылок).

Посмотрим на несколько элементов `info`:

In [30]:
info[10:12]

[('Европа обогнала остальной мир по\xa0темпам нагревания',
  'Марина Попова',
  '2022-11-02',
  '3.1',
  'Экология и климат',
  'Это данные доклада Всемирной метеорологической организации по парниковым газам Европа получила статус самого быстро нагревающегося региона мира: в течение последних 30 лет она нагревалась на 0,5 градуса Цельсия каждые 10 лет, то есть в среднем в два раза интенсивнее, чем другие регионы. Ущерб ее экономике от экстремальных климатических событий в 2021 году превысил 50 миллиардов долларов. Люди и экосистемы в регионе пострадали от волн жары, наводнений, засух и масштабных природных пожаров. Вместе с тем Европа достигла определенных успехов в снижении антропогенных выбросов парниковых газов. К таким выводам пришли авторы доклада Всемирной метеорологической организации и Европейской программы дистанционного зондирования Земли Copernicus о состоянии климата в Европе в 2021 году, пресс-релиз которого поступил в редакцию N + 1. Всемирная метеорологическая организаци

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [31]:
import pandas as pd

In [32]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Население Южной Америки разделилось на две гру...,Михаил Подрезов,2022-11-03,10:02,"03.11.22, 3.9, Антропология, Археология","Похоже, это произошло в андском регионе Палеог..."
1,Хиральность фононов в киновари оказалась истинной,Марат Хамадеев,2022-11-03,17:11,"03.11.22, 8.2, Физика",А связь между их угловым и псевдоугловым момен...
2,Маленькие геймеры отличились высоким уровнем р...,Анастасия Ляшенко,2022-11-03,16:19,"03.11.22, 3.2, Психология",Результаты не зависели от пола ребенка Америка...
3,Прием кофеина во время беременности отнял пару...,Сергей Задворьев,2022-11-03,15:46,"03.11.22, 2.5, Медицина","Даже в восемь лет дети от матерей, получавших ..."
4,Машинное обучение помогло найти оптимальные ус...,Михаил Бойм,2022-11-03,15:25,"03.11.22, 7.1, Химия",Выход описанных в литературе реакций повысился...


Не везде информация сгрузилась корректно, есть проблемы унификации, можно потом поправить это, написав функцию и применив ее через `.apply()` к соответствующим столбцам.

А пока добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [33]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]
df.to_excel("nplus1.xlsx")